In [ ]:
# Install necessary packages
!pip install qiskit
#!pip install qutip==4.7.4

import numpy as np
import qiskit as qi
#import qutip as Q
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, losses, metrics, Input
from tensorflow.keras.utils import plot_model
from sklearn import metrics as sklearn_metrics  # Renamed sklearn's metrics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries for file upload and data manipulation
from google.colab import files
import pandas as pd
import numpy as np

# Upload the Kaggle API key (kaggle.json) using the "files" module
files.upload()

# Install the Kaggle library
!pip install -q kaggle

# Create the ~/.kaggle directory and copy the Kaggle API key into it
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle using the Kaggle API
!kaggle datasets download -d alikookani/quantangle1

# Create a directory to store the downloaded dataset (e.g., 'train')
!mkdir train

# Unzip the downloaded dataset ('quantangle1.zip') into the 'train' directory
!unzip quantangle1.zip -d train

# Read the data from a CSV file ('mydata.csv') into a Pandas DataFrame
data = pd.read_csv('/content/train/mydata.csv')

# Convert the DataFrame to a NumPy array for further processing
data = (np.array(data)).astype(complex)

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/alikookani/quantangle1
License(s): ODbL-1.0
 98% 199M/203M [00:02<00:00, 101MB/s] 
100% 203M/203M [00:02<00:00, 96.2MB/s]
Archive:  quantangle1.zip
  inflating: train/mydata.csv        


In [ ]:
# Number of samples from each class
Samples = 500000
# Shuffle the dataset
data_temp = np.empty([len(data), 16], dtype=complex)
reg_label = np.empty([len(data)], dtype=np.float32)
class_label = np.ones(len(data)).astype('int')
for i in range(Samples):
    data_temp[2 * i, :] = data[i, 0:16]
    reg_label[2 * i] = data[i, 17].real
    class_label[2 * i] = 0
    data_temp[2 * i + 1, :] = data[i + Samples, 0:16]
    reg_label[2 * i + 1] = data[i + Samples, 17].real

In [ ]:
class_label = tf.keras.utils.to_categorical(class_label)

In [ ]:
sigma_z = qi.quantum_info.Pauli('Z').to_matrix()
sigma_x = qi.quantum_info.Pauli('X').to_matrix()
sigma_y = qi.quantum_info.Pauli('Y').to_matrix()
identity = (np.eye(2)).astype(complex)
IX = np.kron(identity, sigma_x)
IY = np.kron(identity, sigma_y)
IZ = np.kron(identity, sigma_z)
XI = np.kron(sigma_x, identity)
XX = np.kron(sigma_x, sigma_x)
XY = np.kron(sigma_x, sigma_y)
XZ = np.kron(sigma_x, sigma_z)
YI = np.kron(sigma_y, identity)
YX = np.kron(sigma_y, sigma_x)
YY = np.kron(sigma_y, sigma_y)
YZ = np.kron(sigma_y, sigma_z)
ZI = np.kron(sigma_z, identity)
ZX = np.kron(sigma_z, sigma_x)
ZY = np.kron(sigma_z, sigma_y)
ZZ = np.kron(sigma_z, sigma_z)

In [ ]:
# Separating real and imaginary values
data_temp = data_temp.reshape(len(data), 16)
vector2 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector2[i, 2 * j] = data_temp[i, j].real
        vector2[i, 2 * j + 1] = data_temp[i, j].imag
# Reshaping to 3D tensors
tensor = vector2.reshape(len(data), 4, 4, 2)

In [ ]:
#XpookyNet Model

input_tensor = Input(shape=(4, 4, 2))
layer1 = layers.Conv2D(64, 1)(input_tensor)
#layer1 = layers.BatchNormalization()(layer1)
layer1 = layers.LeakyReLU()(layer1)
layer2 = layers.Conv2D(64, 1)(layer1)
#layer2 = layers.BatchNormalization()(layer2)
layer2 = layers.LeakyReLU()(layer2)
branch_a = layers.Conv2D(256, 4)(layer2)
#branch_a = layers.BatchNormalization()(branch_a)
branch_a = layers.LeakyReLU()(branch_a)
branch_a = layers.Conv2D(128, 1)(branch_a)
#branch_a = layers.BatchNormalization()(branch_a)
branch_a = layers.LeakyReLU()(branch_a)
branch_b = layers.Conv2D(192, 3)(layer2)
#branch_b = layers.BatchNormalization()(branch_b)
branch_b = layers.LeakyReLU()(branch_b)
layer3 = layers.Conv2D(128, 2)(layer2)
#layer3 = layers.BatchNormalization()(layer3)
layer3 = layers.LeakyReLU()(layer3)
layer4 = layers.Conv2D(128, 1)(layer3)
#layer4 = layers.BatchNormalization()(layer4)
layer4 = layers.LeakyReLU()(layer4)
branch_c = layers.Conv2D(96, 3)(layer4)
#branch_c = layers.BatchNormalization()(branch_c)
branch_c = layers.LeakyReLU()(branch_c)
layer5 = layers.Conv2D(128, 2)(layer4)
#layer5 = layers.BatchNormalization()(layer5)
layer5 = layers.LeakyReLU()(layer5)
layer5 = layers.concatenate([layer5, branch_b], axis=-1)
layer6 = layers.Conv2D(256, 1)(layer5)
#layer6 = layers.BatchNormalization()(layer6)
layer6 = layers.LeakyReLU()(layer6)
layer7 = layers.Conv2D(256, 2)(layer6)
#layer7 = layers.BatchNormalization()(layer7)
layer7 = layers.LeakyReLU()(layer7)
layer7 = layers.concatenate([layer7, branch_a], axis=-1)
layer8 = layers.Conv2D(512, 1)(layer7)
#layer8 = layers.BatchNormalization()(layer8)
layer8 = layers.LeakyReLU()(layer8)
layer8 = layers.concatenate([layer8, branch_c], axis=-1)
layer9 = layers.Conv2D(512, 1)(layer8)
#layer9 = layers.BatchNormalization()(layer9)
layer9 = layers.LeakyReLU()(layer9)
layer10 = layers.Conv2D(512, 1)(layer9)
#layer10 = layers.BatchNormalization()(layer10)
layer10 = layers.LeakyReLU()(layer10)
top1 = layers.Flatten()(layer10)
top1 = layers.Dense(512)(top1)
#top1 = layers.BatchNormalization()(top1)
top1 = layers.LeakyReLU()(top1)

#top2 = layers.Dense(1, name='regression')(top1)

top3 = layers.Dense(2)(top1)
#top3 = layers.BatchNormalization()(top3)
top3 = layers.Activation('softmax', name='classification')(top3)

#model = models.Model(input_tensor, [top2, top3])
model = models.Model(input_tensor, top3)
#model = models.Model(input_tensor, top2)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 4, 4, 2)]            0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 4, 4, 64)             192       ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 4, 4, 64)             0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 4, 4, 64)             4160      ['leaky_re_lu[0][0]']         
                                                                                              

In [ ]:
# Compile the model
callback_list = [
                 tf.keras.callbacks.ModelCheckpoint(
                     filepath = 'two_Qbit_LessTomogrophy.keras',
                     monitor = 'val_loss',
                     save_best_only = True,
                 )
]

model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

Epoch 1/4
7840/7840 [==============================] - 71s 8ms/step - loss: 0.3016 - categorical_accuracy: 0.8559 - val_loss: 0.1742 - val_categorical_accuracy: 0.9248
Epoch 2/4
7840/7840 [==============================] - 66s 8ms/step - loss: 0.1472 - categorical_accuracy: 0.9355 - val_loss: 0.1259 - val_categorical_accuracy: 0.9432
Epoch 3/4
7840/7840 [==============================] - 64s 8ms/step - loss: 0.1137 - categorical_accuracy: 0.9506 - val_loss: 0.1309 - val_categorical_accuracy: 0.9435
Epoch 4/4
7840/7840 [==============================] - 63s 8ms/step - loss: 0.1001 - categorical_accuracy: 0.9567 - val_loss: 0.1008 - val_categorical_accuracy: 0.9570


In [ ]:
#First manual reduction in learning rate
model.compile(
    optimizer=optimizers.SGD(
        learning_rate=0.001, momentum=0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history2 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 3,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

Epoch 1/3
7840/7840 [==============================] - 69s 8ms/step - loss: 0.0475 - categorical_accuracy: 0.9809 - val_loss: 0.0506 - val_categorical_accuracy: 0.9786
Epoch 2/3
7840/7840 [==============================] - 65s 8ms/step - loss: 0.0411 - categorical_accuracy: 0.9827 - val_loss: 0.0509 - val_categorical_accuracy: 0.9777
Epoch 3/3
7840/7840 [==============================] - 64s 8ms/step - loss: 0.0386 - categorical_accuracy: 0.9838 - val_loss: 0.0513 - val_categorical_accuracy: 0.9789


In [ ]:
#Second manual reduction in learning rate
model.compile(
    optimizer=optimizers.SGD(
        learning_rate=0.0001, momentum=0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history3 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 2,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

Epoch 1/2
7840/7840 [==============================] - 68s 8ms/step - loss: 0.0288 - categorical_accuracy: 0.9888 - val_loss: 0.0424 - val_categorical_accuracy: 0.9823
Epoch 2/2
7840/7840 [==============================] - 64s 8ms/step - loss: 0.0277 - categorical_accuracy: 0.9892 - val_loss: 0.0434 - val_categorical_accuracy: 0.9813


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 2s 5ms/step - loss: 0.0379 - categorical_accuracy: 0.9850


[0.03792877867817879, 0.9850000143051147]

In [ ]:
data_temp = data_temp.reshape(len(data), 4, 4) #length data and data_temp are the same so used the shorter name

In [ ]:
IX_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_removed[i, :, :] = (data_temp[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), IX))/4 * IX)

In [ ]:
IX_removed = IX_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_removed[i, j].imag
# Reshaping to 3D tensors
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 2,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

Epoch 1/2
7840/7840 [==============================] - 67s 8ms/step - loss: 0.1785 - categorical_accuracy: 0.9290 - val_loss: 0.1648 - val_categorical_accuracy: 0.9343
Epoch 2/2
7840/7840 [==============================] - 64s 8ms/step - loss: 0.1640 - categorical_accuracy: 0.9351 - val_loss: 0.1586 - val_categorical_accuracy: 0.9372


In [ ]:
#First manual reduction in learning rate
model.compile(
    optimizer=optimizers.SGD(
        learning_rate=0.001, momentum=0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history2 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.1414 - categorical_accuracy: 0.9471 - val_loss: 0.1447 - val_categorical_accuracy: 0.9449


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 3ms/step - loss: 0.1503 - categorical_accuracy: 0.9429


[0.1503078192472458, 0.9429000020027161]

In [ ]:
IX_removed = IX_removed.reshape([len(data_temp), 4, 4])
IX_IY_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_removed[i, :, :] = IX_removed[i, :, :] - (np.trace(np.matmul((data_temp[i, :, :]), IY))/4 * IY)

In [ ]:
IX_IY_removed = IX_IY_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_removed[i, j].imag
# Reshaping to 3D tensors
del IX_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.2237 - categorical_accuracy: 0.9058 - val_loss: 0.2146 - val_categorical_accuracy: 0.9082


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 4ms/step - loss: 0.2281 - categorical_accuracy: 0.9021


[0.22813740372657776, 0.9021000266075134]

In [ ]:
IX_IY_removed = IX_IY_removed.reshape([len(data_temp), 4, 4])
IX_IY_XI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_XI_removed[i, :, :] = (IX_IY_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), XI))/4 * XI)

In [ ]:
IX_IY_XI_removed = IX_IY_XI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_XI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_XI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.2455 - categorical_accuracy: 0.8934 - val_loss: 0.2416 - val_categorical_accuracy: 0.8972


In [ ]:
#First manual reduction in learning rate
model.compile(
    optimizer=optimizers.SGD(
        learning_rate=0.001, momentum=0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history2 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.2319 - categorical_accuracy: 0.8994 - val_loss: 0.2349 - val_categorical_accuracy: 0.9002


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 2s 6ms/step - loss: 0.2426 - categorical_accuracy: 0.8939


[0.2426401674747467, 0.8938999772071838]

In [ ]:
IX_IY_XI_removed = IX_IY_XI_removed.reshape([len(data_temp), 4, 4])
IX_IY_XI_YI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_XI_YI_removed[i, :, :] = (IX_IY_XI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), YI))/4 * YI)

In [ ]:
IX_IY_XI_YI_removed = IX_IY_XI_YI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_XI_YI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_XI_YI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_XI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 68s 8ms/step - loss: 0.2656 - categorical_accuracy: 0.8810 - val_loss: 0.2668 - val_categorical_accuracy: 0.8816


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 2s 5ms/step - loss: 0.2761 - categorical_accuracy: 0.8710


[0.27607059478759766, 0.8709999918937683]

In [ ]:
IX_IY_XI_YI_removed = IX_IY_XI_YI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_YI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_YI_removed[i, :, :] = (IX_IY_XI_YI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), IZ))/4 * IZ)

In [ ]:
IX_IY_IZ_XI_YI_removed = IX_IY_IZ_XI_YI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_YI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_YI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_XI_YI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.2915 - categorical_accuracy: 0.8661 - val_loss: 0.2918 - val_categorical_accuracy: 0.8680


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 4ms/step - loss: 0.3022 - categorical_accuracy: 0.8592


[0.3022049069404602, 0.8592000007629395]

In [ ]:
IX_IY_IZ_XI_YI_removed = IX_IY_IZ_XI_YI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_YI_ZI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_YI_ZI_removed[i, :, :] = (IX_IY_IZ_XI_YI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), ZI))/4 * ZI)

In [ ]:
IX_IY_IZ_XI_YI_ZI_removed = IX_IY_IZ_XI_YI_ZI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_YI_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_YI_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_YI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.3018 - categorical_accuracy: 0.8600 - val_loss: 0.3066 - val_categorical_accuracy: 0.8561


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 4ms/step - loss: 0.3145 - categorical_accuracy: 0.8496


[0.3145008087158203, 0.8496000170707703]

In [ ]:
IX_IY_IZ_XI_YI_ZI_removed = IX_IY_IZ_XI_YI_ZI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_YI_ZI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_YI_ZI_removed[i, :, :] = (IX_IY_IZ_XI_YI_ZI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), XX))/4 * XX)

In [ ]:
IX_IY_IZ_XI_XX_YI_ZI_removed = IX_IY_IZ_XI_XX_YI_ZI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_YI_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_YI_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_YI_ZI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.3672 - categorical_accuracy: 0.8274 - val_loss: 0.3697 - val_categorical_accuracy: 0.8263


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 2s 5ms/step - loss: 0.3708 - categorical_accuracy: 0.8241


[0.37078189849853516, 0.8241000175476074]

In [ ]:
IX_IY_IZ_XI_XX_YI_ZI_removed = IX_IY_IZ_XI_XX_YI_ZI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_YI_ZI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_YI_ZI_removed[i, :, :] = (IX_IY_IZ_XI_XX_YI_ZI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), XY))/4 * XY)

In [ ]:
IX_IY_IZ_XI_XX_XY_YI_ZI_removed = IX_IY_IZ_XI_XX_XY_YI_ZI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_YI_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_YI_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_YI_ZI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 69s 8ms/step - loss: 0.4217 - categorical_accuracy: 0.7964 - val_loss: 0.4217 - val_categorical_accuracy: 0.7988


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 3ms/step - loss: 0.4290 - categorical_accuracy: 0.7900


[0.42896366119384766, 0.7900000214576721]

In [ ]:
IX_IY_IZ_XI_XX_XY_YI_ZI_removed = IX_IY_IZ_XI_XX_XY_YI_ZI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed[i, :, :] = (IX_IY_IZ_XI_XX_XY_YI_ZI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), YX))/4 * YX)

In [ ]:
IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed = IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_YI_ZI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 68s 8ms/step - loss: 0.4782 - categorical_accuracy: 0.7592 - val_loss: 0.4781 - val_categorical_accuracy: 0.7596


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 4ms/step - loss: 0.4868 - categorical_accuracy: 0.7519


[0.48682549595832825, 0.7519000172615051]

In [ ]:
IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed = IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed[i, :, :] = (IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), YY))/4 * YY)

In [ ]:
IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed = IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_YI_YX_ZI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.5377 - categorical_accuracy: 0.7213 - val_loss: 0.5353 - val_categorical_accuracy: 0.7236


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 3ms/step - loss: 0.5450 - categorical_accuracy: 0.7193


[0.5449782013893127, 0.7192999720573425]

In [ ]:
IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed = IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed[i, :, :] = (IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), XZ))/4 * XZ)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_YI_YX_YY_ZI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.5705 - categorical_accuracy: 0.6973 - val_loss: 0.5682 - val_categorical_accuracy: 0.7046


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 2s 4ms/step - loss: 0.5740 - categorical_accuracy: 0.6967


[0.5740314722061157, 0.6966999769210815]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed[i, :, :] = (IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), YZ))/4 * YZ)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_ZI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.6062 - categorical_accuracy: 0.6733 - val_loss: 0.6038 - val_categorical_accuracy: 0.6789


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 2s 5ms/step - loss: 0.6059 - categorical_accuracy: 0.6747


[0.6059061884880066, 0.6747000217437744]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed[i, :, :] = (IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), ZX))/4 * ZX)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.6314 - categorical_accuracy: 0.6450 - val_loss: 0.6325 - val_categorical_accuracy: 0.6418


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 3ms/step - loss: 0.6326 - categorical_accuracy: 0.6383


[0.6325736045837402, 0.6383000016212463]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed[i, :, :] = (IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), ZY))/4 * ZY)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.6581 - categorical_accuracy: 0.6067 - val_loss: 0.6543 - val_categorical_accuracy: 0.6167


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 1s 4ms/step - loss: 0.6613 - categorical_accuracy: 0.5993


[0.6612839102745056, 0.5993000268936157]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed.reshape([len(data_temp), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_ZZ_removed = np.empty([len(data_temp), 4, 4], dtype=complex)
for i in range(len(data_temp)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_ZZ_removed[i, :, :] = (IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed[i, :, :]) - (np.trace(np.matmul((data_temp[i, :, :]), ZZ))/4 * ZZ)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_ZZ_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_ZZ_removed.reshape(len(data_temp), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(data), 32], dtype=np.float32)
for i in range(len(data)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_ZZ_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_ZZ_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_removed
tensor = vector1.reshape(len(data), 4, 4, 2)

In [ ]:
np.set_printoptions(edgeitems=5, threshold=20, linewidth=80, formatter={'all':lambda x: f"{x:.4f}"})


In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YY_YZ_ZI_ZX_ZY_ZZ_removed[0, :].reshape(4, 4)

array([[0.2500+0.0000j, 0.0000+0.0000j, 0.0000-0.0000j, 0.0000+0.0000j],
       [0.0000+0.0000j, 0.2500+0.0000j, 0.0000+0.0000j, 0.0000-0.0000j],
       [0.0000+0.0000j, 0.0000+0.0000j, 0.2500+0.0000j, 0.0000+0.0000j],
       [0.0000-0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.2500+0.0000j]])

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(data)-20000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': class_label[:len(data)-20000]
     },
    batch_size = 125,
    epochs = 1,
    callbacks = callback_list,
    validation_data = (tensor[len(data)-20000:len(data)-10000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': class_label[len(data)-20000:len(data)-10000]
                      }
                     ),
)

7840/7840 [==============================] - 67s 8ms/step - loss: 0.6934 - categorical_accuracy: 0.5001 - val_loss: 0.6931 - val_categorical_accuracy: 0.5000


In [ ]:
# Evaluate the model using TensorFlow/Keras metrics
model.evaluate(tensor[len(data)-10000:len(data), :, :, :], #{'regression': reg_label[len(data)-10000:len(data)],
                                                     #'classification':
                    class_label[len(data)-10000:len(data)]
                    #}
                    )

313/313 [==============================] - 2s 6ms/step - loss: 0.6931 - categorical_accuracy: 0.5000


[0.6931466460227966, 0.5]

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Define the save path
save_path = '/gdrive/My Drive/MyFolder/Tomography.keras'

# Ensure the folder exists, if not, create it
import os
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the model
model.save(save_path)

In [ ]:
from tensorflow.keras.models import load_model

# Path to the saved model in Google Drive
model_path = '/gdrive/My Drive/MyFolder/Tomography.keras'

# Load the model
model = load_model(model_path)

# Verify the model is loaded
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 4, 4, 2)]            0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 4, 4, 64)             192       ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 4, 4, 64)             0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 4, 4, 64)             4160      ['leaky_re_lu[0][0]']         
                                                                                              

In [ ]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=ae67d4f7ab5aa99cd62bb968eb66318708d56bb4c753fe704c87d4d4cd7d1fc0
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, DensityMatrix

# Create a quantum circuit
n_qubits = 2
data_amount = 10000
density_matrices = np.empty([data_amount, 4, 4], dtype=complex)
for i in range(data_amount):
  qc = QuantumCircuit(n_qubits)
  # Apply a U gate with random parameters to each qubit
  for qubit in range(n_qubits):
      theta = np.random.uniform(0, 2 * np.pi)
      phi = np.random.uniform(0, 2 * np.pi)
      lambda_ = np.random.uniform(0, 2 * np.pi)
      qc.u(theta, phi, lambda_, qubit)
  if (i % 4) == 0:
    qc.h(0)
    qc.cx(0, 1)
  elif (i % 4) == 2:
    qc.h(1)
    qc.cx(1, 0)
  # Get the statevector of the quantum circuit
  statevector = Statevector.from_instruction(qc)

  # Convert the statevector to a density matrix
  density_matrix = DensityMatrix(statevector)
  probs = Statevector(statevector).probabilities_dict()

  density_matrices[i, :, :] = np.array(density_matrix.data, dtype=complex)

In [ ]:
alternating_list = [1 if i % 2 == 0 else 0 for i in range(len(density_matrices))]
label_array = np.array(alternating_list, dtype=int)

In [ ]:
label_array = tf.keras.utils.to_categorical(label_array)

In [ ]:
IX_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_removed[i, :, :] = (density_matrices[i, :, :]) - (np.trace(np.matmul((density_matrices[i, :, :]), IX))/4 * IX)

In [ ]:
IX_removed = IX_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_removed[i, j].imag
# Reshaping to 3D tensors
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 22s 304ms/step - loss: 0.0931 - categorical_accuracy: 0.9645 - val_loss: 0.0339 - val_categorical_accuracy: 0.9930
Epoch 2/4
64/64 [==============================] - 19s 299ms/step - loss: 0.0246 - categorical_accuracy: 0.9940 - val_loss: 0.0305 - val_categorical_accuracy: 0.9940
Epoch 3/4
64/64 [==============================] - 19s 303ms/step - loss: 0.0151 - categorical_accuracy: 0.9965 - val_loss: 0.0262 - val_categorical_accuracy: 0.9930
Epoch 4/4
64/64 [==============================] - 20s 316ms/step - loss: 0.0119 - categorical_accuracy: 0.9970 - val_loss: 0.0334 - val_categorical_accuracy: 0.9910


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 9ms/step - loss: 0.0122 - categorical_accuracy: 0.9970


[0.012181775644421577, 0.996999979019165]

In [ ]:
IX_removed = IX_removed.reshape([len(density_matrices), 4, 4])
IX_IY_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_removed[i, :, :] = IX_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), IY))/4 * IY)

In [ ]:
IX_IY_removed = IX_IY_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_removed[i, j].imag
# Reshaping to 3D tensors
del IX_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 22s 315ms/step - loss: 0.0244 - categorical_accuracy: 0.9929 - val_loss: 0.0278 - val_categorical_accuracy: 0.9940
Epoch 2/4
64/64 [==============================] - 19s 301ms/step - loss: 0.0091 - categorical_accuracy: 0.9976 - val_loss: 0.0283 - val_categorical_accuracy: 0.9940
Epoch 3/4
64/64 [==============================] - 21s 322ms/step - loss: 0.0076 - categorical_accuracy: 0.9980 - val_loss: 0.0249 - val_categorical_accuracy: 0.9950
Epoch 4/4
64/64 [==============================] - 19s 299ms/step - loss: 0.0072 - categorical_accuracy: 0.9979 - val_loss: 0.0257 - val_categorical_accuracy: 0.9960


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 10ms/step - loss: 0.0104 - categorical_accuracy: 0.9970


[0.010356473736464977, 0.996999979019165]

In [ ]:
IX_IY_removed = IX_IY_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_removed[i, :, :] = IX_IY_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), IZ))/4 * IZ)

In [ ]:
IX_IY_IZ_removed = IX_IY_IZ_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 23s 332ms/step - loss: 0.0824 - categorical_accuracy: 0.9809 - val_loss: 0.0291 - val_categorical_accuracy: 0.9930
Epoch 2/4
64/64 [==============================] - 19s 299ms/step - loss: 0.0156 - categorical_accuracy: 0.9959 - val_loss: 0.0252 - val_categorical_accuracy: 0.9950
Epoch 3/4
64/64 [==============================] - 20s 306ms/step - loss: 0.0096 - categorical_accuracy: 0.9974 - val_loss: 0.0231 - val_categorical_accuracy: 0.9960
Epoch 4/4
64/64 [==============================] - 20s 312ms/step - loss: 0.0066 - categorical_accuracy: 0.9979 - val_loss: 0.0283 - val_categorical_accuracy: 0.9940


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 16ms/step - loss: 0.0111 - categorical_accuracy: 0.9970


[0.01108618825674057, 0.996999979019165]

In [ ]:
IX_IY_IZ_removed = IX_IY_IZ_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_removed[i, :, :] = IX_IY_IZ_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), XI))/4 * XI)

In [ ]:
IX_IY_IZ_XI_removed = IX_IY_IZ_XI_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 21s 306ms/step - loss: 0.0094 - categorical_accuracy: 0.9973 - val_loss: 0.0255 - val_categorical_accuracy: 0.9950
Epoch 2/4
64/64 [==============================] - 19s 302ms/step - loss: 0.0065 - categorical_accuracy: 0.9976 - val_loss: 0.0248 - val_categorical_accuracy: 0.9960
Epoch 3/4
64/64 [==============================] - 21s 322ms/step - loss: 0.0039 - categorical_accuracy: 0.9989 - val_loss: 0.0264 - val_categorical_accuracy: 0.9960
Epoch 4/4
64/64 [==============================] - 21s 320ms/step - loss: 0.0031 - categorical_accuracy: 0.9994 - val_loss: 0.0288 - val_categorical_accuracy: 0.9960


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 11ms/step - loss: 0.0095 - categorical_accuracy: 0.9970


[0.009469522163271904, 0.996999979019165]

In [ ]:
IX_IY_IZ_XI_removed = IX_IY_IZ_XI_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_removed[i, :, :] = IX_IY_IZ_XI_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), XX))/4 * XX)

In [ ]:
IX_IY_IZ_XI_XX_removed = IX_IY_IZ_XI_XX_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 21s 303ms/step - loss: 0.0706 - categorical_accuracy: 0.9789 - val_loss: 0.0572 - val_categorical_accuracy: 0.9860
Epoch 2/4
64/64 [==============================] - 20s 319ms/step - loss: 0.0327 - categorical_accuracy: 0.9889 - val_loss: 0.0446 - val_categorical_accuracy: 0.9900
Epoch 3/4
64/64 [==============================] - 19s 301ms/step - loss: 0.0245 - categorical_accuracy: 0.9924 - val_loss: 0.0446 - val_categorical_accuracy: 0.9900
Epoch 4/4
64/64 [==============================] - 19s 301ms/step - loss: 0.0210 - categorical_accuracy: 0.9930 - val_loss: 0.0457 - val_categorical_accuracy: 0.9920


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 10ms/step - loss: 0.0349 - categorical_accuracy: 0.9880


[0.03488384187221527, 0.9879999756813049]

In [ ]:
IX_IY_IZ_XI_XX_removed = IX_IY_IZ_XI_XX_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_removed[i, :, :] = IX_IY_IZ_XI_XX_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), XY))/4 * XY)

In [ ]:
IX_IY_IZ_XI_XX_XY_removed = IX_IY_IZ_XI_XX_XY_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 21s 308ms/step - loss: 0.0451 - categorical_accuracy: 0.9839 - val_loss: 0.0434 - val_categorical_accuracy: 0.9880
Epoch 2/4
64/64 [==============================] - 20s 309ms/step - loss: 0.0278 - categorical_accuracy: 0.9902 - val_loss: 0.0519 - val_categorical_accuracy: 0.9890
Epoch 3/4
64/64 [==============================] - 21s 320ms/step - loss: 0.0204 - categorical_accuracy: 0.9945 - val_loss: 0.0412 - val_categorical_accuracy: 0.9910
Epoch 4/4
64/64 [==============================] - 19s 303ms/step - loss: 0.0183 - categorical_accuracy: 0.9941 - val_loss: 0.0491 - val_categorical_accuracy: 0.9910


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 9ms/step - loss: 0.0345 - categorical_accuracy: 0.9890


[0.034479424357414246, 0.9890000224113464]

In [ ]:
IX_IY_IZ_XI_XX_XY_removed = IX_IY_IZ_XI_XX_XY_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), XZ))/4 * XZ)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_removed = IX_IY_IZ_XI_XX_XY_XZ_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 23s 324ms/step - loss: 0.0198 - categorical_accuracy: 0.9923 - val_loss: 0.0341 - val_categorical_accuracy: 0.9940
Epoch 2/4
64/64 [==============================] - 19s 301ms/step - loss: 0.0153 - categorical_accuracy: 0.9949 - val_loss: 0.0306 - val_categorical_accuracy: 0.9940
Epoch 3/4
64/64 [==============================] - 19s 302ms/step - loss: 0.0149 - categorical_accuracy: 0.9936 - val_loss: 0.0310 - val_categorical_accuracy: 0.9940
Epoch 4/4
64/64 [==============================] - 20s 317ms/step - loss: 0.0154 - categorical_accuracy: 0.9940 - val_loss: 0.0455 - val_categorical_accuracy: 0.9890


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 10ms/step - loss: 0.0368 - categorical_accuracy: 0.9890


[0.036826081573963165, 0.9890000224113464]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_removed = IX_IY_IZ_XI_XX_XY_XZ_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), YI))/4 * YI)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 23s 326ms/step - loss: 0.0374 - categorical_accuracy: 0.9891 - val_loss: 0.0493 - val_categorical_accuracy: 0.9910
Epoch 2/4
64/64 [==============================] - 19s 302ms/step - loss: 0.0204 - categorical_accuracy: 0.9931 - val_loss: 0.0336 - val_categorical_accuracy: 0.9950
Epoch 3/4
64/64 [==============================] - 19s 302ms/step - loss: 0.0142 - categorical_accuracy: 0.9944 - val_loss: 0.0363 - val_categorical_accuracy: 0.9940
Epoch 4/4
64/64 [==============================] - 20s 310ms/step - loss: 0.0126 - categorical_accuracy: 0.9955 - val_loss: 0.0353 - val_categorical_accuracy: 0.9960


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 12ms/step - loss: 0.0242 - categorical_accuracy: 0.9930


[0.024195872247219086, 0.9929999709129333]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_YI_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), YX))/4 * YX)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 22s 321ms/step - loss: 0.1314 - categorical_accuracy: 0.9630 - val_loss: 0.0823 - val_categorical_accuracy: 0.9800
Epoch 2/4
64/64 [==============================] - 20s 312ms/step - loss: 0.0597 - categorical_accuracy: 0.9830 - val_loss: 0.0431 - val_categorical_accuracy: 0.9910
Epoch 3/4
64/64 [==============================] - 19s 299ms/step - loss: 0.0453 - categorical_accuracy: 0.9868 - val_loss: 0.0500 - val_categorical_accuracy: 0.9890
Epoch 4/4
64/64 [==============================] - 19s 301ms/step - loss: 0.0428 - categorical_accuracy: 0.9869 - val_loss: 0.0431 - val_categorical_accuracy: 0.9890


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 15ms/step - loss: 0.0577 - categorical_accuracy: 0.9820


[0.05769799277186394, 0.9819999933242798]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), YZ))/4 * YZ)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 21s 305ms/step - loss: 0.3026 - categorical_accuracy: 0.9029 - val_loss: 0.1574 - val_categorical_accuracy: 0.9430
Epoch 2/4
64/64 [==============================] - 19s 300ms/step - loss: 0.1472 - categorical_accuracy: 0.9491 - val_loss: 0.1469 - val_categorical_accuracy: 0.9520
Epoch 3/4
64/64 [==============================] - 21s 323ms/step - loss: 0.1198 - categorical_accuracy: 0.9582 - val_loss: 0.1434 - val_categorical_accuracy: 0.9460
Epoch 4/4
64/64 [==============================] - 19s 304ms/step - loss: 0.1224 - categorical_accuracy: 0.9569 - val_loss: 0.1123 - val_categorical_accuracy: 0.9660


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 10ms/step - loss: 0.1227 - categorical_accuracy: 0.9560


[0.12274611741304398, 0.9559999704360962]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), ZI))/4 * ZI)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 23s 327ms/step - loss: 0.7031 - categorical_accuracy: 0.7215 - val_loss: 0.4470 - val_categorical_accuracy: 0.7700
Epoch 2/4
64/64 [==============================] - 19s 297ms/step - loss: 0.4110 - categorical_accuracy: 0.7934 - val_loss: 0.4259 - val_categorical_accuracy: 0.7820
Epoch 3/4
64/64 [==============================] - 19s 303ms/step - loss: 0.3876 - categorical_accuracy: 0.8021 - val_loss: 0.4102 - val_categorical_accuracy: 0.7930
Epoch 4/4
64/64 [==============================] - 20s 315ms/step - loss: 0.3713 - categorical_accuracy: 0.8154 - val_loss: 0.4157 - val_categorical_accuracy: 0.7890


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 10ms/step - loss: 0.3826 - categorical_accuracy: 0.8020


[0.38256600499153137, 0.8019999861717224]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), ZX))/4 * ZX)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 23s 315ms/step - loss: 0.3648 - categorical_accuracy: 0.8198 - val_loss: 0.3879 - val_categorical_accuracy: 0.8150
Epoch 2/4
64/64 [==============================] - 19s 301ms/step - loss: 0.3588 - categorical_accuracy: 0.8219 - val_loss: 0.3845 - val_categorical_accuracy: 0.7970
Epoch 3/4
64/64 [==============================] - 19s 303ms/step - loss: 0.3517 - categorical_accuracy: 0.8248 - val_loss: 0.3849 - val_categorical_accuracy: 0.8160
Epoch 4/4
64/64 [==============================] - 20s 319ms/step - loss: 0.3471 - categorical_accuracy: 0.8291 - val_loss: 0.3804 - val_categorical_accuracy: 0.8100


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 12ms/step - loss: 0.3551 - categorical_accuracy: 0.8220


[0.3550804853439331, 0.8220000267028809]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), ZY))/4 * ZY)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 21s 307ms/step - loss: 0.3938 - categorical_accuracy: 0.7880 - val_loss: 0.4300 - val_categorical_accuracy: 0.7740
Epoch 2/4
64/64 [==============================] - 19s 303ms/step - loss: 0.3806 - categorical_accuracy: 0.7922 - val_loss: 0.4021 - val_categorical_accuracy: 0.7850
Epoch 3/4
64/64 [==============================] - 20s 318ms/step - loss: 0.3768 - categorical_accuracy: 0.7972 - val_loss: 0.4189 - val_categorical_accuracy: 0.7630
Epoch 4/4
64/64 [==============================] - 19s 302ms/step - loss: 0.3712 - categorical_accuracy: 0.7995 - val_loss: 0.4058 - val_categorical_accuracy: 0.7830


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 10ms/step - loss: 0.3809 - categorical_accuracy: 0.7930


[0.38086777925491333, 0.7929999828338623]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), ZZ))/4 * ZZ)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 22s 312ms/step - loss: 0.7689 - categorical_accuracy: 0.6130 - val_loss: 0.5961 - val_categorical_accuracy: 0.6880
Epoch 2/4
64/64 [==============================] - 21s 320ms/step - loss: 0.5891 - categorical_accuracy: 0.6939 - val_loss: 0.5901 - val_categorical_accuracy: 0.6880
Epoch 3/4
64/64 [==============================] - 19s 298ms/step - loss: 0.5850 - categorical_accuracy: 0.6938 - val_loss: 0.5883 - val_categorical_accuracy: 0.6930
Epoch 4/4
64/64 [==============================] - 19s 301ms/step - loss: 0.5859 - categorical_accuracy: 0.6889 - val_loss: 0.5825 - val_categorical_accuracy: 0.6910


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 13ms/step - loss: 0.6110 - categorical_accuracy: 0.6670


[0.6110350489616394, 0.6669999957084656]

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed.reshape([len(density_matrices), 4, 4])
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_YY_removed = np.empty([len(density_matrices), 4, 4], dtype=complex)
for i in range(len(density_matrices)):
  IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_YY_removed[i, :, :] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed[i, :, :] - (np.trace(np.matmul((density_matrices[i, :, :]), YY))/4 * YY)

In [ ]:
IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_YY_removed = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_YY_removed.reshape(len(density_matrices), 16)

In [ ]:
# Separating real and imaginary values
vector1 = np.empty([len(density_matrices), 32], dtype=np.float32)
for i in range(len(density_matrices)):
    for j in range(16):
        vector1[i, 2 * j] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_YY_removed[i, j].real
        vector1[i, 2 * j + 1] = IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_YY_removed[i, j].imag
# Reshaping to 3D tensors
del IX_IY_IZ_XI_XX_XY_XZ_YI_YX_YZ_ZI_ZX_ZY_ZZ_removed
tensor = vector1.reshape(len(density_matrices), 4, 4, 2)

In [ ]:
model.compile(
    optimizer = optimizers.SGD(
        learning_rate = 0.01, momentum = 0.9,
    ),
    loss = {
#        'regression': losses.mse
        'classification': losses.categorical_crossentropy
        },
    metrics = {
#        'regression': metrics.mae
        'classification': metrics.categorical_accuracy
        }
)

history1 = model.fit(
    tensor[:len(density_matrices)-2000, :, :, :],
    {#'regression': reg_label[:len(data)-20000]
     'classification': label_array[:len(density_matrices)-2000]
     },
    batch_size = 125,
    epochs = 4,
    callbacks = callback_list,
    validation_data = (tensor[len(density_matrices)-2000:len(density_matrices)-1000, :, :, :],
                     {#'regression': reg_label[len(data)-20000:len(data)-10000]
                      'classification': label_array[len(density_matrices)-2000:len(density_matrices)-1000]
                      }
                     ),
)

Epoch 1/4
64/64 [==============================] - 21s 310ms/step - loss: 0.7036 - categorical_accuracy: 0.4958 - val_loss: 0.6938 - val_categorical_accuracy: 0.5000
Epoch 2/4
64/64 [==============================] - 20s 314ms/step - loss: 0.6942 - categorical_accuracy: 0.4886 - val_loss: 0.6932 - val_categorical_accuracy: 0.5000
Epoch 3/4
64/64 [==============================] - 20s 314ms/step - loss: 0.6932 - categorical_accuracy: 0.5073 - val_loss: 0.6942 - val_categorical_accuracy: 0.5000
Epoch 4/4
64/64 [==============================] - 19s 303ms/step - loss: 0.6941 - categorical_accuracy: 0.4947 - val_loss: 0.6932 - val_categorical_accuracy: 0.5000


In [ ]:
model.evaluate(tensor[len(density_matrices)-1000:len(density_matrices), :, :, :], label_array[len(density_matrices)-1000:len(density_matrices), :])

32/32 [==============================] - 1s 10ms/step - loss: 0.6932 - categorical_accuracy: 0.5000


[0.6932384967803955, 0.5]